# NN-自然語言處理
## 教學目標
- 本教學著重於自然語言處理，主要涵蓋 `RNN`。
- 這份教學的目標是介紹如何以 Python 和 PyTorch 實作神經網路。

## 使用 NN 來進行中文的分類任務

- 我們將在這個教學裡讓大家實作中文情緒分析（Sentiment Analysis）
- 本資料集爲外賣平臺用戶評價分析，[下載連結](https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/waimai_10k/waimai_10k.csv)。
- 資料集欄位爲標籤（label）和評價（review），
- 標籤 1 爲正向，0 爲負向。
- 正向 4000 條，負向約 8000 條。

In [1]:
# 0. 下載資料與安裝 jieba

!mkdir -p data
!wget https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/waimai_10k/waimai_10k.csv -O data/waimai_10k.csv
!pip install jieba

In [2]:
# 1. 導入所需套件

# 第3方套件
import jieba
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

/data/workspace/CGUDL_2025_fall/.venv/lib/python3.12/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# 2. 以 pandas 讀取資料
# 請先下載資料集

df = pd.read_csv("./data/waimai_10k.csv")

In [4]:
# 3. 觀察資料

df.head()

,label,review
0,1,很快，好吃，味道足，量大
1,1,没有送水没有送水没有送水
2,1,非常快，态度好。
3,1,方便，快捷，味道可口，快递给力
4,1,菜味道很棒！送餐很及时！


## 建立字典
- 電腦無法僅透過字符來區分不同字之間的意涵
- 電腦視覺領域依賴的是影像資料本身的像素值
- 我們讓電腦理解文字的方法是透過向量
- 文字的意義藉由向量來進行表達的形式稱為 word embeddings
- 舉例:
$\textrm{apple}=[0.123, 0.456,0.789,\dots,0.111]$

- 如何建立每個文字所屬的向量？
    - 傳統方法: 計數法則
    - 近代方法 (2013-至今): 使用(淺層)神經網路訓練 word2vec ([參考](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/))，稱為 word embeddings
    - 現代方法 (2018-至今): 使用(深層)神經網路訓練 Transformers，也就是BERT ([參考](https://youtu.be/gh0hewYkjgo))，又稱為 contexualized embeddings
- 在那之前，要先建立分散式字詞的字典
    - 可粗分兩種斷詞方式 (tokenization):
        1. 每個字都斷 (character-level)
        2. 斷成字詞 (word-level)

## Word embeddings
- 著名的方法有:
    1. word2vec: Skip-gram, CBOW (continuous bag-of-words)
    2. GloVe
    3. fastText
- 本教學使用 PyTorch 內建的 Embedding 層來實作 word embeddings

In [5]:
word_to_idx = {"<pad>": 0, "<unk>": 1, "好吃": 2, "棒": 3, "给力": 4}
embeds = torch.nn.Embedding(5, 5)  # 5 words in vocab, 5 dimensional embeddings

In [6]:
def get_word_id(word, vocab, unk_idx: int = 1):
    return vocab.get(word, unk_idx)

In [7]:
lookup_tensor = torch.tensor(
    [
        get_word_id("好吃", word_to_idx),
        get_word_id("不棒", word_to_idx),
        get_word_id("<unk>", word_to_idx),
    ],
)
word_embed = embeds(lookup_tensor)
print(word_embed)

tensor([[-1.8612,  0.0705, -0.6171, -0.5732,  0.5950],
        [ 1.4814,  1.0650, -3.2168,  0.4955, -0.1476],
        [ 1.4814,  1.0650, -3.2168,  0.4955, -0.1476]],
       grad_fn=<EmbeddingBackward0>)


### 複習 torch.nn.Linear 用法

In [8]:
# torch.nn.Linear 用法範例
# m 是一個線性轉換層，之前我們都在 forward 裡面使用它

m = torch.nn.Linear(20, 30)
input = torch.randn(128, 20) # 假設有 128 筆資料，每筆資料有 20 維
output = m(input)

print(output.size())

torch.Size([128, 30])


In [9]:
# 4. 建立字典
use_jieba=True

vocab = {'<pad>':0, '<unk>':1}

if use_jieba:
    words = []
    for sent in df['review']:
        tokens = jieba.lcut(sent, cut_all=False)
        words.extend(tokens)

else:
    # 以 character-level 斷詞
    words = df['review'].str.cat()

# 使字詞不重複
words = sorted(set(words))
for idx, word in enumerate(words):
    # 一開始已經放兩個進去 dictionary 了
    idx = idx + 2
    # 將 word to id 放到 dictionary
    vocab[word] = idx

# 查看字典大小
print("The vocab size is {}.".format(len(vocab)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.591 seconds.
Prefix dict has been built successfully.


The vocab size is 11010.


## 使用 PyTorch 建立 Dataset
![Imgur](https://i.imgur.com/wGnfCmH.png)

In [10]:
# 5. 將資料分成 train/ validation/ test

train_data, test_data = train_test_split(
    df,
    test_size=0.2,
)
train_data, validation_data = train_test_split(
    train_data,
    test_size=0.1,
)

In [11]:
# 6. 定義超參數

parameters = {
    "padding_idx": 0,
    "vocab_size": len(vocab),
    # Hyperparameters
    "embed_dim": 300,
    "hidden_dim": 256,
    "module_name": 'rnn', # 選項: rnn, lstm, gru, transformer
    "num_layers": 2,
    "learning_rate": 5e-4, # 使用 Transformer 時建議改成 5e-5
    "epochs": 10,
    "max_seq_len": 50,
    "batch_size": 64,
    "bidirectional": True,
}

In [12]:
# 7. 建立 PyTorch Dataset (定義 class)

class WaimaiDataset(torch.utils.data.Dataset):
    # 繼承 torch.utils.data.Dataset
    def __init__(self, vocab, data, max_seq_len, use_jieba):
        self.df = data
        self.max_seq_len = max_seq_len
        # 可以選擇要不要使用結巴進行斷詞
        self.use_jieba = use_jieba
        self.vocab = vocab
        self.unk_idx = self.vocab.get('<unk>')

    # 改寫繼承的 __getitem__ function
    def __getitem__(self, idx):
        # dataframe 的第一個 column 是 label
        # dataframe 的第一個 column 是 評論的句子
        label, sent = self.df.iloc[idx, 0:2]
        # 先將 label 轉為 float32 以方便後面進行 loss function 的計算
        label_tensor = torch.tensor(label, dtype=torch.float32)
        if self.use_jieba:
            # 使用 lcut 可以 return list
            tokens = jieba.lcut(sent, cut_all=False)
        else:
            # 每個字都斷詞
            tokens = list(sent)

        # 控制最大的序列長度
        tokens = tokens[:self.max_seq_len]

        # 根據 vocab 轉換 word id
        # 如果找不到該字詞，就用 <unk> 的 index 來表示
        tokens_id = [self.vocab.get(word, self.unk_idx) for word in tokens]
        tokens_tensor = torch.LongTensor(tokens_id)

        # 所以 第 0 個index是句子，第 1 個index是 label
        return tokens_tensor, label_tensor

    # 改寫繼承的 __len__ function
    def __len__(self):
        return len(self.df)

In [13]:
# 8. 建立 PyTorch Dataset (執行 class)
use_jieba=use_jieba

trainset = WaimaiDataset(
    vocab,
    train_data,
    parameters["max_seq_len"],
    use_jieba=use_jieba
)
validset = WaimaiDataset(
    vocab,
    validation_data,
    parameters["max_seq_len"],
    use_jieba=use_jieba
)
testset = WaimaiDataset(
    vocab,
    test_data,
    parameters["max_seq_len"],
    use_jieba=use_jieba
)

In [14]:
# 9. 整理 batch 的資料 (定義 function)

def collate_batch(batch):
    # 抽每一個 batch 的第 0 個(注意順序)
    text = [i[0] for i in batch]
    # 進行 padding
    text = pad_sequence(text, batch_first=True)

    # 抽每一個 batch 的第 1 個(注意順序)
    label = [i[1] for i in batch]
    # 把每一個 batch 的答案疊成一個 tensor
    label = torch.stack(label)

    return text, label

In [15]:
# 10. 建立資料分批 (mini-batches)

# 因為會針對 trainloader 進行 shuffle
# 對 trainloader 進行 shuffle 有助於降低 overfitting

trainloader = DataLoader(
    trainset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=True,
)
validloader = DataLoader(
    validset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=False,
)
testloader = DataLoader(
    testset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=False,
)

## 建立模型
![Imgur](https://i.imgur.com/OgLBBm7.png)
- 模型建置的流程如上圖所示
- 文字的部份會透過 Dataset 及 DataLoader 進行處理
- embedding 層經由 nn.embedding 來實現 embedding lookup 的功能
- embedding 層再接上模型，最後接上分類層，即可進行分類任務
- 本範例提供的 Model class 可以藉由更換 module_name 來呼叫不同的 RNN

In [16]:
# 11. 建立 RNN 模型 (定義 class)

class RNNModel(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, padding_idx, bi=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.bidirectional = bi

        # 定義 Embedding 層
        self.embedding = torch.nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim,
            padding_idx=padding_idx
        )
        # 定義 LSTM
        self.rnn = torch.nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=bi
        )
        # 根據是否 bidirectional 決定輸出層的輸入維度
        direction_factor = 2 if bi else 1
        self.fc = torch.nn.Linear(
            # 如果 bi-directional，hidden_dim 是兩倍
            in_features=hidden_dim * direction_factor,
            out_features=1
        )

    def forward(self, X):
        """定義神經網路的前向傳遞的進行流程
        Arguments:
            - X: 輸入值，維度為(B, S)，其中 B 為 batch size，S 為 sentence length
        Returns:
            - logits: 模型的輸出值，維度為(B, 1)，其中 B 為 batch size
            - Y: 模型的輸出值但經過非線性轉換 (這邊是用 sigmoid)，維度為(B, 1)，其中 B 為 batch size
        """
        # 維度: (B, S) -> (B, S, E)
        # B: batch size; S: sentence length; E: embedding dimension
        E = self.embedding(X)

        # 使用 RNN 系列
        H_out, (h_n, c_n) = self.rnn(E)

        if self.bidirectional:
            # 取第一個和最後一個 hidden states做相加 (bi-directional)
            combined = torch.cat([H_out[:, -1, :self.hidden_dim],
                                H_out[:, 0, self.hidden_dim:]], dim=1)
        else:
            # 取最後一個 hidden states (uni-directional)
            combined = H_out[:, -1, :]

        logits = self.fc(combined)
        Y = torch.sigmoid(logits)

        return logits, Y

In [17]:
# 12. 執行訓練所需要的準備工作

model = RNNModel(
    vocab_size=parameters["vocab_size"],
    embed_dim=parameters["embed_dim"],
    hidden_dim=parameters["hidden_dim"],
    padding_idx=parameters["padding_idx"],
    bi=parameters["bidirectional"],
)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=parameters["learning_rate"])
loss_func = torch.nn.BCEWithLogitsLoss() # 含有 sigmoid 的版本

## 設定訓練流程

In [18]:
# 13. 設定訓練流程 (定義 function)

def train(trainloader, model, optimizer, loss_func):
    """定義訓練時的進行流程
    Arguments:
        - trainloader: 具備 mini-batches 的 dataset，由 PyTorch DataLoader 所建立
        - model: 要進行訓練的模型
        - optimizer: 最佳化目標函數的演算法
    Returns:
        - train_loss: 模型在一個 epoch 的 training loss
    """
    # 設定模型的訓練模式
    model.train()

    # 記錄一個 epoch中 training 過程的 loss
    train_loss = 0
    # 從 trainloader 一次一次抽
    for x, y in tqdm(trainloader, desc="Training"):
        # 將變數丟到指定的裝置位置
        x = x.to(device)
        y = y.to(device)

        # 重新設定模型的梯度
        optimizer.zero_grad()

        # 1. 前向傳遞 (Forward Pass)
        logits, pred = model(x)

        # 2. 計算 loss (loss function 為二元交叉熵)
        loss = loss_func(logits.squeeze(-1), y)

        # 3. 計算反向傳播的梯度
        loss.backward()
        # 4. "更新"模型的權重
        optimizer.step()

        # 一個 epoch 會抽很多次 batch，所以每個 batch 計算完都要加起來
        # .item() 在 PyTorch 中可以獲得該 tensor 的數值
        train_loss += loss.item()

    return train_loss / len(trainloader)

## 設定驗證流程

In [19]:
# 14. 設定驗證流程 (定義 function)

def evaluate(dataloader, model, loss_func):
    """定義驗證時的進行流程
    Arguments:
        - dataloader: 具備 mini-batches 的 dataset，由 PyTorch DataLoader 所建立
        - model: 要進行驗證的模型
    Returns:
        - loss: 模型在驗證/測試集的 loss
        - acc: 模型在驗證/測試集的正確率
    """
    # 設定模型的驗證模式
    # 此時 dropout 會自動關閉
    model.eval()
    total_loss = 0 # 紀錄 loss 數值
    label_list = []
    prediction_list = []

    # 設定現在不計算梯度
    with torch.no_grad():
        # 從 dataloader 一次一次抽
        for x, y in tqdm(dataloader, desc="Evaluating"):
            x, y = x.to(device), y.to(device)
            logits, pred = model(x)

            # 計算 loss (loss function 為二元交叉熵)
            # 模型輸出的維度是 (B, 1)，使用.squeeze(-1)可以讓維度變 (B,)
            loss = loss_func(logits.squeeze(-1), y)
            total_loss += loss.item()

            # 預測的數值大於 0.5 則視為類別1，反之為類別0
            pred = (pred > 0.5) * 1 # pred.shape: (B, 1)
            prediction_list.extend(pred.cpu().squeeze(-1).tolist())
            label_list.extend(y.cpu().tolist())
    
    avg_loss = total_loss / len(dataloader)
    # 計算正確率
    acc = accuracy_score(label_list, prediction_list)

    return avg_loss, acc

## 開始訓練

In [20]:
# 15. 整個訓練及驗證過程的 script

train_loss_history = []
valid_loss_history = []

for epoch in range(parameters["epochs"]):
    train_loss = train(
        trainloader,
        model,
        optimizer=optimizer,
        loss_func=loss_func,
    )

    print("Training loss at epoch {} is {}.".format(epoch+1, train_loss))
    train_loss_history.append(train_loss)

    if epoch % 2 == 1:
        print("=====Start validation=====")
        valid_loss, valid_acc = evaluate(
            dataloader=validloader,
            model=model,
            loss_func=loss_func,
        )
        valid_loss_history.append(valid_loss)
        print("Validation accuracy at epoch {} is {}, and validation loss is {}."\
              .format(epoch+1, valid_acc, valid_loss))

    torch.save(model.state_dict(), "model_epoch_{}.pkl".format(epoch))

Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:02<00:00, 46.75it/s]


Training loss at epoch 1 is 0.4261981639597151.


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:02<00:00, 50.16it/s]


Training loss at epoch 2 is 0.2865349610646566.
=====Start validation=====


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 63.97it/s]


...

Training loss at epoch 9 is 0.05956658816861886.


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:02<00:00, 49.89it/s]


Training loss at epoch 10 is 0.07818729467689992.
=====Start validation=====


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 65.17it/s]

Validation accuracy at epoch 10 is 0.8644421272158499, and validation loss is 0.39072684148947395.


In [21]:
# 16. 預測測試集

best_epoch = np.argmin(valid_loss_history)
model.load_state_dict(
    torch.load("model_epoch_{}.pkl".format(best_epoch))
)

print("=====Start testing=====")
test_loss, test_acc = evaluate(testloader, model, loss_func)
print("Testing accuracy is {}.".format(test_acc))

=====Start testing=====


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 63.95it/s]

Testing accuracy is 0.8603002502085071.
